# Import Google Drive + Darknet Github

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

root_dir = "/content/gdrive/MyDrive"
base_dir = root_dir + '/Colab Notebooks/Cars'

Mounted at /content/gdrive


In [ ]:
DN = "/content/darknet"
!git clone https://github.com/AlexeyAB/darknet $DN # Makes a darknet folder

Cloning into '/content/darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 12.29 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.


# Set compiler up (Say if there is a GPU)

In [ ]:
# https://stackoverflow.com/questions/39086/search-and-replace-a-line-in-a-file-in-python

f = open(DN + '/Makefile', 'r')
lines = ''
for line in f:
    if line.find('GPU=0') != -1:
        line = line.replace('GPU=0', 'GPU=1')
    if line.find('CUDNN=0') != -1:
       line = line.replace('CUDNN=0', 'CUDNN=1')
    if line.find('CUDNN_HALF=0') != -1:
       line = line.replace('CUDNN_HALF=0', 'CUDNN_HALF=1')
    if line.find('OPENCV=0') != -1:
        line = line.replace('OPENCV=0', 'OPENCV=1')
    if line.find('AVX=0') != -1:
        line = line.replace('AVX=0', 'AVX=1')
    if line.find('OPENMP=0') != -1:
        line = line.replace('OPENMP=0', 'OPENMP=1')
    lines += line
f.close()

g = open('/content/darknet/Makefile', 'w')
g.write(lines)
g.close()

# Compile Darknet

In [ ]:
!cd $DN; make --silent; clear; echo "Darknet Compiled!"

./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘buff’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
 1443 |         char buff[100];
      |              ^~~~
./src/image_opencv.cpp:1419:9: warning: unused variable ‘it_tb_res’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
 1419 |     int it_tb_res = cv::createTrackbar(it_trackbar_name, window_name, 

# Set Darknet Up


    Batches = number of classes * 2000
    Filters = (number of classes + 5) * 3


In [ ]:
!cp $DN/cfg/yolov3.cfg $DN/cfg/yolov3-train.cfg

f = open(DN + '/cfg/yolov3-train.cfg', 'r')
lines = ''
for line in f:
    if line.find('batch=1') != -1:
        line = line.replace('batch=1', 'batch=64')
    if line.find('subdivisions=1') != -1:
        line = line.replace('subdivisions=1', 'subdivisions=16')
    if line.find('max_batches=500200') != -1:
        line = line.replace('max_batches=500200', 'max_batches=2000')
    if line.find('filters=255') != -1:
        line = line.replace('filters=255', 'filters=18')
    if line.find('classes=80') != -1:
        line = line.replace('classes=80', 'classes=1')
    lines += line
f.close()
with open(DN + '/cfg/yolov3-train.cfg', 'w') as f:
  f.write(lines)

In [ ]:
!cd $DN/data; rm * -fr
# r = recursive, f = force (and shut up)

!unzip -q "$base_dir/cars.zip" -d "$DN/data"
# q = Quiet, d = Target Directory

In [ ]:
!mkdir $root_dir/yolo-license-plates
!echo -e "classes = 1\ntrain = $DN/data/train.txt\nvalid = $DN/data/test.txt\nlabels = $DN/data/labels.txt\nnames = $DN/data/cars/classes.txt\nbackup = $root_dir/yolo-license-plates" > $DN/data/obj.data

mkdir: cannot create directory ‘/content/gdrive/MyDrive/yolo-license-plates’: File exists


In [ ]:
!find $DN/data/cars/images/*.jpeg > $DN/data/train.txt
!find $DN/data/cars/images/*.jpeg > $DN/data/test.txt

!mv $DN/data/cars/labels/*.txt $DN/data/cars/images
!find $DN/data/cars/images/*.txt > $DN/data/labels.txt

In [ ]:
!cd $DN; wget https://pjreddie.com/media/files/darknet53.conv.74
#cd $DŃ: wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2024-03-26 14:41:06--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  17.7MB/s    in 9.6s    

2024-03-26 14:41:17 (16.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# Run Darknet

In [ ]:
!date

Tue Mar 26 02:41:22 PM UTC 2024


In [ ]:
#!cd $DN;./darknet detector train $DN/data/obj.data $DN/cfg/yolov3-train.cfg /content/darknet/darknet53.conv.74 -dont_show
!cd $DN; ./darknet detector -dont_show -map train $DN/data/obj.data $DN/cfg/yolov3-train.cfg $DN/darknet53.conv.74
!date

 GPU isn't used 
 Used AVX 
 Used FMA & AVX2 
 OpenCV version: 4.5.4
 Prepare additional network for mAP calculation...
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 

In [ ]:
from IPython.display import display

display("Hello World")
print('Not to be confused with print(Hello World) since these don\'t allow copying.')

'Hello World'

Not to be confused with print(Hello World) since these don't allow copying.


It outputs a ton with mixed estimations of how many hours it will take.